In [ ]:
# ! pip install -U "autogen-agentchat" "autogen-ext[openai]" 

In [ ]:
# ! pip install -U "autogen-ext[semantic-kernel-hugging-face]"

'''
error 
'''

# from autogen_ext.models.semantic_kernel import SKChatCompletionAdapter
# from semantic_kernel import Kernel
# from semantic_kernel.connectors.ai.hugging_face import HuggingFaceTextCompletion,HuggingFacePromptExecutionSettings

# sk_client = HuggingFaceTextCompletion(
#     ai_model_id=ONLINE_MODEL_NAME,
#     api_key=api_key
# )

# settings = HuggingFacePromptExecutionSettings(
#     temperature=0.2,
# )

# anthropic_model_client = SKChatCompletionAdapter(
#     sk_client, kernel=Kernel(), prompt_settings=settings
# )  

In [ ]:
import os
import sys
try:
    get_ipython
    current_dir = os.getcwd()
except NameError:
    current_dir = os.path.dirname(os.path.abspath(__file__))

# Set path，temporary path expansion
project_dir = os.path.abspath(os.path.join(current_dir, '..'))
if project_dir not in sys.path:
    sys.path.append(project_dir)
    
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
import asyncio
from tools.tool import skip_execution
IS_SKIP =True

In [ ]:
load_dotenv()
BASE_URL= os.getenv("BASE_URL")
ONLINE_MODEL_NAME = os.getenv("ONLINE_MODEL_NAME")
api_key = ( os.getenv("QWEN_API_KEY") or os.getenv("DASHSCOPE_API_KEY" ) ) 

model_info = {
    "name": f"{ONLINE_MODEL_NAME}", # 模型名称，可随意填写
    "parameters": {
        "max_tokens": 2048,  # 每次输出最大token数
        "temperature": 0.4,  
        "top_p": 0.9,  
    },
    "family": "unknown",  # 必填字段，model属于的类别
    "functions": [],  # 非必填字段，如果模型支持函数调用，可以在这里定义函数信息
    "vision": False,  # 必填字段，模型是否支持图像输入
    "json_output": True,  # 必填字段，模型是否支持json格式输出
    "function_calling": True,  # 必填字段，模型是否支持函数调用，如果模型需要使用工具函数，该字段为true
    "structured_output":True
}


# # Define a model client. You can use other model client that implements
# # the `ChatCompletionClient` interface.
model_client = OpenAIChatCompletionClient(
    model=ONLINE_MODEL_NAME,
    api_key=api_key,
    base_url=BASE_URL,
    model_info= model_info
)


In [ ]:
@skip_execution(IS_SKIP)
async def test_model_connected():
    '''
    测试模型是否联通
    '''
    from autogen_core.models import UserMessage

    result = await model_client.create([UserMessage(content="What is the capital of China?", source="user")])
    print(result)
    await model_client.close()
    
await test_model_connected()

异步函数 test_model_connected 已跳过执行


In [ ]:

# Define a simple function tool that the agent can use.
# For this example, we use a fake weather tool for demonstration purposes.
async def get_weather(city: str) -> str:
    """Get the weather for a given city."""
    return f"The weather in {city} is 73 degrees and Sunny."


# Define an AssistantAgent with the model, tool, system message, and reflection enabled.
# The system message instructs the agent via natural language.
agent = AssistantAgent(
    name="weather_agent",
    model_client=model_client,
    tools=[get_weather],
    system_message="You are a helpful assistant.",
    reflect_on_tool_use=False,
    model_client_stream=False,  # Enable streaming tokens from the model client.
)


# Run the agent and stream the messages to the console.
async def main() -> None:
    await Console(agent.run_stream(task="What is the weather in New York?"))
    # Close the connection to the model client.
    await model_client.close()


# NOTE: if running this inside a Python script you'll need to use asyncio.run(main()).
await main()

---------- TextMessage (user) ----------
What is the weather in New York?
---------- ToolCallRequestEvent (weather_agent) ----------
[FunctionCall(id='call_ec2ec20ae733442da12ee3', arguments='{"city": "New York"}', name='get_weather')]
---------- ToolCallExecutionEvent (weather_agent) ----------
[FunctionExecutionResult(content='The weather in New York is 73 degrees and Sunny.', name='get_weather', call_id='call_ec2ec20ae733442da12ee3', is_error=False)]
---------- ToolCallSummaryMessage (weather_agent) ----------
The weather in New York is 73 degrees and Sunny.
